In [138]:
#This notebook is a prototype for a script that will splice individual
#UFUDs into UFSD files. 

#Ellie White, 26 Jan 2021

%matplotlib inline
import numpy as np
import pylab as plt
import sys
np.set_printoptions(threshold= sys.maxsize)

#first, read in the .dat files for the first scan in the cadence:
#Need to figure out how to make the filenames more general or at least 
#how to make them work for running on the GBO machine

nodes_list = ['00', '01', '02', '03', '04', '05', '06', '11', \
              '12', '13', '14', '15', '16', '21', '22', '23', \
              '24', '25', '26', '27']

filenames_list = []

for node in nodes_list:
    filename = '/datax/scratch/ewhite/voyager-xband/dat_files/BLC{0}/fine_res/'\
               'blc{0}_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.dat'.format(node)
    filenames_list.append(filename)

#these arrays store data for plotting histograms
dat_info = []
header_lines = []

for fname in filenames_list:
    infile = open(fname, 'r')
    file_header = []

    for line in infile.readlines():
        if line[0] != '#':
            dat_row = line.split()
            
            #get rid of unnecessary whitespace that might
            #cause problems with sorting
            for i in range(len(dat_row)):
                dat_row[i] = dat_row[i].strip()
                
            #append each row to the 2-d dat_info array
            dat_info.append(dat_row)
        else:
            file_header.append(line)
    header_lines.append(file_header)

num_rows = len(dat_info)
dat_info = np.array(dat_info, float)

#sort the rows so they are in order of frequency, descending
freq_index = 4
sorted_dat_info = dat_info[dat_info[:,freq_index].argsort()[::-1]]

#replace Total_Hit_# values so they are unique for each
#row and are natural numbers, matching the turboSETI convention
total_hit_nums = np.arange(1, num_rows+1, 1)
sorted_dat_info[:,0] = total_hit_nums

#address coarse channel numbers
sorted_coarse_chans = []
chan_num = 0

for c in range(num_rows):
    if c == 0:
        sorted_coarse_chans.append(chan_num)
    
    #if there are multiple rows for one coarse channel, the coarse
    #channel number needs to stay the same for all of those rows:
    elif sorted_dat_info[c,10] == sorted_dat_info[(c-1),10]:
        sorted_coarse_chans.append(chan_num)
        
    #if your coarse channel is different from the last one, it needs to be incremented:
    else:
        chan_num += 1
        sorted_coarse_chans.append(chan_num)

sorted_dat_info[:,10] = sorted_coarse_chans

#write spliced .dat file header

outfile_name = '/home/ewhite/ufsd_test.dat'

outfile = open(o_fname, 'w')
outfile.write(header_lines[0][0])

for header in header_lines:
    outfile.write(header[1])

outfile.write(header_lines[0][2]+header_lines[0][3]+header_lines[0][4]+header_lines[0][5]+header_lines[0][6]+header_lines[0][7]+header_lines[0][8])

#write rows for spliced .dat file

for r in range(num_rows):
#    outfile.write('{:<{width}}{: }\trow\n'.format(total_hit_nums[r], sorted_dat_info[r,1], width=space_padding))
    
    info_str = '%03d\t'%(int(sorted_dat_info[r,0]))  #Top Hit number
    info_str += '%10.6f\t'%(float(sorted_dat_info[r,1]))  #Drift Rate
    info_str += '%10.6f\t'%(float(sorted_dat_info[r,2]))  #SNR
    info_str += '%14.6f\t'%(float(sorted_dat_info[r,3])) #Uncorrected Frequency:
    info_str += '%14.6f\t'%(float(sorted_dat_info[r,4])) #Corrected Frequency:
    info_str += '%d\t'%(int(sorted_dat_info[r,5])) #Index:
    info_str += '%14.6f\t'%(float(sorted_dat_info[r,6])) #freq_start:
    info_str += '%14.6f\t'%(float(sorted_dat_info[r,7])) #freq_end:
    info_str += '%s\t'%(float(sorted_dat_info[r,8])) #SEFD:
    info_str += '%14.6f\t'%(float(sorted_dat_info[r,9])) #SEFD_mid_freq:
    info_str += '%i\t'%(int(sorted_dat_info[r,10]))
    info_str += '%i\t'%(int(sorted_dat_info[r,11])) #
    info_str +='\n'
    
    outfile.write(info_str)
    
outfile.close()

#get columns from SFD for comparison: (this part can be uncommented if you want to compare
#the columns of our UFSD with the columns of an equivalent SFD)
sfd_fname = '/datax/scratch/ewhite/voyager-xband/spliced_guppi_59046_80036_DIAG_VOYAGER-1_0011.rawspec.0000.dat'
sfd_infile = open(sfd_fname, 'r')

s_dat_info = []

for s_line in sfd_infile.readlines():
    if s_line[0] != '#':
        s_items = s_line.split()

        for num in range(len(s_items)):
            s_items[num] = s_items[num].strip()
        
        s_dat_info.append(s_items)
        
s_dat_info = np.array(s_dat_info, float)
s_sorted_dat_info = s_dat_info[s_dat_info[:,freq_index].argsort()[::-1]]
s_sorted_dat_info[:,0] = total_hit_nums
print(np.array_equal(s_sorted_dat_info[:,2], sorted_dat_info[:,2]))

print(sorted_dat_info[:,2])
print()
print(s_sorted_dat_info[:,2])

False
[3.94019204e+02 3.26012215e+03 3.06570003e+02 1.65739814e+05
 5.29110505e+03 6.45412313e+02 2.70987340e+01 2.97064180e+01
 4.19680140e+01 8.21896990e+01 5.96057650e+01 2.79301640e+01
 7.66044244e+08 2.58474510e+01 4.62650920e+01 8.21461210e+01
 4.07289540e+01 3.03292620e+01 2.67755010e+01 6.76385775e+02
 5.61868322e+03 2.46240432e+05 3.08525278e+02 3.56875400e+03
 3.94019277e+02 2.50934213e+06 1.19871648e+03 1.44804623e+03
 2.94063790e+01 5.00859950e+01 7.38620620e+01 1.93732084e+02
 1.02973320e+06 2.61924282e+03 3.55160900e+01 7.99760485e+05
 2.49394978e+04 2.86840178e+02 2.86166826e+03 9.75277360e+05
 2.27729168e+03 6.64346091e+02 9.81525997e+05 2.80207200e+04
 4.15905950e+01 3.26457560e+01 3.20635280e+01 6.70287790e+01
 3.99636922e+04 1.33506794e+03 6.46230970e+01 4.92022900e+01
 5.90010060e+01 4.10382360e+01 9.25035137e+05 3.69550930e+01
 5.06082710e+01 4.53272572e+03 5.61043883e+02 2.25092057e+02
 9.34895408e+02 2.41448909e+03 8.80622882e+03 8.53801917e+05
 1.25241775e+04 3.